In [1]:
pip install transformers #==4.11.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.6 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 6.6 MB 39.0 MB/s 
     |████████████████████████████████| 596 kB 56.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
pip install datasets #==2.2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 4.3 MB/s 
     |████████████████████████████████| 1.1 MB 56.0 MB/s 
     |████████████████████████████████| 212 kB 51.8 MB/s 
     |████████████████████████████████| 140 kB 51.7 MB/s 
     |████████████████████████████████| 127 kB 62.2 MB/s 
     |████████████████████████████████| 271 kB 36.9 MB/s 
     |████████████████████████████████| 144 kB 60.6 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import json
import transformers
from transformers import AutoTokenizer, DataCollatorForLanguageModeling, TrainingArguments, Trainer, AutoModelForMaskedLM
import datasets
from datasets import Features, Value, Sequence, load_metric, load_dataset
import torch

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path='/content/drive/MyDrive/ApResearch/'
dataset = load_dataset('json',data_files={'train':path+'reldatatrain.json', 'test': path+'reldatatest.json'}, field='data')

Using custom data configuration default-8c57e7d9b206646a


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-8c57e7d9b206646a/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"
device = torch.device(dev)

In [13]:
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-pretrain")
#tokenizer.pad_token = tokenizer.eos_token
special_tokens_dict = {'additional_special_tokens': ['[E1]','[E2]','[/E1]','[/E2]']}
num_added_toks=tokenizer.add_special_tokens(special_tokens_dict)
block_size = 512
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

def label(examples):
  examples["labels"] = examples["input_ids"].copy()
  #examples=examples.remove_columns_(examples.column_names)
  return examples

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

def tokenize_inputs(examples):
    return tokenizer(examples["inputs"], is_split_into_words=True, truncation=True, max_length=512, padding='max_length')#, truncation=True, padding='max_length') 
    #return tokenizer([" ".join(x) for x in examples["inputs"]], truncation=True, padding=True)

datatrain = dataset['train'].map(tokenize_inputs, batched=True,remove_columns=dataset['train'].column_names)
datatest = dataset['test'].map(tokenize_inputs, batched=True,remove_columns=dataset['test'].column_names)
datatrain = datatrain.map(label, batched=True)
datatest = datatest.map(label, batched=True)



  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
#torch.cuda.set_device(1)
#torch.cuda.current_device()

In [21]:
model = AutoModelForMaskedLM.from_pretrained("yiyanghkust/finbert-pretrain")
model.resize_token_embeddings(model.config.vocab_size + 4)
#model.to(device)


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datatrain,
    eval_dataset=datatest,
    data_collator=data_collator,
)

trainer.train()

Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 41

Epoch,Training Loss,Validation Loss
1,2.368000,1.331470


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1037
  Batch size = 8


Epoch,Training Loss,Validation Loss
1,2.368000,1.331470
2,1.596800,1.046904
3,1.396000,0.917136


Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1037
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1037
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1557, training_loss=1.7708721369141252, metrics={'train_runtime': 925.1389, 'train_samples_per_second': 13.444, 'train_steps_per_second': 1.683, 'total_flos': 3273755025991680.0, 'train_loss': 1.7708721369141252, 'epoch': 3.0})

In [22]:
torch.save(model.state_dict(), path+"fine_tuned")